In [5]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
#nltk.download('punkt')

# 加载预训练的BERT模型和分词器
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

# 定义文本分类器
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

# 示例Jira描述内容
description = """
Hi team,

Could you please help to look at the case described below to improve the
material report performance?

The query took 20 minutes or more to get the data for one month.

Open image-20240402-052748.png
image-20240402-052748.png
DB: medicalsanmhk-pts.42-q.com / PTSIII_E85

2024-04-01 02:48:24 EDT [31917]: [10-1] db=PTSIII_E85,user=ptswriter,app=[unknown],client=10.191.8.52 LOG:  duration: 501045.690 ms  execute <unnamed>: SELECT                            comp.created_time,comp.barcode,inc.incoming_number,inc.coo,pn.pn_id,pn.part_number,comp.total_qty,comp.current_qty,comp.lot_code,comp.remark,comp.component_id,usr.usr_name,comp.expire_time,aml.manufacturer_id,aml.manufacturer_pn,comp.pb_category, comp.status_id,comp.lpn,aml.aml_id,aml.status_id AS aml_status_id,aml.last_update,                                              inc.incoming_id,comp.manu_info_id,comp.date_code_str,comp.date_code,co.company_name,comp.msd_level,inc.supplier_id,com.company_name

                                                as supplier_name,com.vendor_code, comp.updated_date

                                                FROM

                                                part_number pn,component_1

                                                comp,aml,company

                                                co,user_info usr,incoming inc LEFT

                                                JOIN company com

                                                ON

                                                inc.supplier_id=com.company_id

                                                WHERE

                                                pn.pn_id=aml.pn_id

                                                AND

                                                aml.aml_id=inc.aml_id

                                                AND

                                                aml.manufacturer_id=co.company_id

                                                AND

                                                usr.usr_id=comp.usr_id

                                                AND

 inc.incoming_id=comp.incoming_id  AND comp.created_time > '1709251200'   AND comp.created_time < '1712016000'  ORDER BY comp.created_time DESC OFFSET 0 LIMIT 50000

2024-04-01 03:18:49 EDT [31913]: [10-1] db=PTSIII_E85,user=ptswriter,app=[unknown],client=10.191.8.52 ERROR: canceling statement due to user request

2024-04-01 03:18:49 EDT [31913]: [11-1] db=PTSIII_E85,user=ptswriter,app=[unknown],client=10.191.8.52 STATEMENT:   SELECT COUNT(*) AS total FROM ( SELECT                                 comp.created_time,comp.barcode,inc.incoming_number,inc.coo,pn.pn_id,pn.part_number,comp.total_qty,comp.current_qty,comp.lot_code,comp.remark,comp.component_id,usr.usr_name,comp.expire_time,aml.manufacturer_id,aml.manufacturer_pn,comp.pb_category, comp.status_id,comp.lpn,aml.aml_id,aml.status_id AS aml_status_id,aml.last_update,                                         inc.incoming_id,comp.manu_info_id,comp.date_code_str,comp.date_code,co.company_name,comp.msd_level,inc.supplier_id,com.company_name

                                                as supplier_name,com.vendor_code, comp.updated_date

                                                FROM

                                                part_number pn,component_1

                                                comp,aml,company

                                                co,user_info usr,incoming inc LEFT

                                                JOIN company com

                                                ON

                                                inc.supplier_id=com.company_id

                                                WHERE

                                                pn.pn_id=aml.pn_id

                                                AND

                                                aml.aml_id=inc.aml_id

                                                AND

                                                aml.manufacturer_id=co.company_id

                                                AND

                                                usr.usr_id=comp.usr_id

                                                AND

inc.incoming_id=comp.incoming_id  AND comp.created_time > '1709251200'   AND comp.created_time < '1712016000'  ORDER BY comp.created_time DESC ) AS totalSQLView
"""

description2 = """
Hello team,



Could you help us to check the TLA Serial Count Report? The ‚ÄúMonths‚Äú value is being displayed as 2024, instead of the past months:

!image-20240110-203955.png|width=827,height=578!





The query used is taking the current year to add it:



mesr_aws=# select  
date_trunc('month',now()) ||'to'|| date_trunc('month',now() + INTERVAL '1 months') - INTERVAL '1 second' as sort_
,to_char( (now() ), 'Month') || ' ' || extract( year from (now() )) as date_
union all
select  
date_trunc('month',now() - INTERVAL '1 months') ||'to'|| date_trunc('month',now() + INTERVAL '0 months') - INTERVAL '1 second'
,to_char((now() - INTERVAL '1 months' ), 'Month') || ' ' || extract( year from (now() )) as date_
union all
select  
date_trunc('month',now() - INTERVAL '2 months') ||'to'|| date_trunc('month',now() - INTERVAL '1 months') - INTERVAL '1 second'
,to_char((now() - INTERVAL '2 months' ), 'Month') || ' ' || extract( year from (now() )) as date_
union all
select  
date_trunc('month',now() - INTERVAL '3 months') ||'to'|| date_trunc('month',now() - INTERVAL '2 months') - INTERVAL '1 second'
,to_char((now() - INTERVAL '3 months' ), 'Month') || ' ' || extract( year from (now() )) as date_
union all
select  
date_trunc('month',now() - INTERVAL '4 months') ||'to'|| date_trunc('month',now() - INTERVAL '3 months') - INTERVAL '1 second'
,to_char((now() - INTERVAL '4 months' ), 'Month') || ' ' || extract( year from (now() )) as date_
union all
select  
date_trunc('month',now() - INTERVAL '5 months') ||'to'|| date_trunc('month',now() - INTERVAL '4 months') - INTERVAL '1 second'
,to_char((now() - INTERVAL '5 months' ), 'Month') || ' ' || extract( year from (now() )) as date_
union all
select  
date_trunc('month',now() - INTERVAL '6 months') ||'to'|| date_trunc('month',now() - INTERVAL '5 months') - INTERVAL '1 second'
,to_char((now() - INTERVAL '6 months' ), 'Month') || ' ' || extract( year from (now() )) as date_
union all
select  
date_trunc('month',now() - INTERVAL '7 months') ||'to'|| date_trunc('month',now() - INTERVAL '6 months') - INTERVAL '1 second'
,to_char((now() - INTERVAL '7 months' ), 'Month') || ' ' || extract( year from (now() )) as date_
union all
select  
date_trunc('month',now() - INTERVAL '8 months') ||'to'|| date_trunc('month',now() - INTERVAL '7 months') - INTERVAL '1 second'
,to_char((now() - INTERVAL '8 months' ), 'Month') || ' ' || extract( year from (now() )) as date_
union all
select  
date_trunc('month',now() - INTERVAL '9 months') ||'to'|| date_trunc('month',now() - INTERVAL '8 months') - INTERVAL '1 second'
,to_char((now() - INTERVAL '9 months' ), 'Month') || ' ' || extract( year from (now() )) as date_
union all
select  
date_trunc('month',now() - INTERVAL '10 months') ||'to'|| date_trunc('month',now() - INTERVAL '9 months') - INTERVAL '1 second'
,to_char((now() - INTERVAL '10 months' ), 'Month') || ' ' || extract( year from (now() )) as date_
union all
select  
date_trunc('month',now() - INTERVAL '11 months') ||'to'|| date_trunc('month',now() - INTERVAL '10 months') - INTERVAL '1 second'
,to_char((now() - INTERVAL '11 months' ), 'Month') || ' ' || extract( year from (now() )) as date_ ;
                     sort_                      |     date_      
------------------------------------------------+----------------
 2024-01-01 00:00:00+00to2024-01-31 23:59:59+00 | January   2024
 2023-12-01 00:00:00+00to2023-12-31 23:59:59+00 | December  2024
 2023-11-01 00:00:00+00to2023-11-30 23:59:59+00 | November  2024
 2023-10-01 00:00:00+00to2023-10-31 23:59:59+00 | October   2024
 2023-09-01 00:00:00+00to2023-09-30 23:59:59+00 | September 2024
 2023-08-01 00:00:00+00to2023-08-31 23:59:59+00 | August    2024
 2023-07-01 00:00:00+00to2023-07-31 23:59:59+00 | July      2024
 2023-06-01 00:00:00+00to2023-06-30 23:59:59+00 | June      2024
 2023-05-01 00:00:00+00to2023-05-31 23:59:59+00 | May       2024
 2023-04-01 00:00:00+00to2023-04-30 23:59:59+00 | April     2024
 2023-03-01 00:00:00+00to2023-03-31 23:59:59+00 | March     2024
 2023-02-01 00:00:00+00to2023-02-28 23:59:59+00 | February  2024
(12 rows)

mesr_aws=# 





Please let us know if further informatio is needed.









Thank you,

Regards.
"""

# 分句处理
sentences = description.split('\n')

# 过滤掉无意义的句子
def filter_sentences(sentences, classifier):
    meaningful_sentences = []
    for sentence in sentences:
        if len(sentence.strip()) > 0:
            result = classifier(sentence)
            if result[0]['label'] == 'LABEL_1':  # 假设LABEL_1表示有意义的句子
                meaningful_sentences.append(sentence)
    return meaningful_sentences

# 过滤掉无意义的句子

sql_pattern = r"(SELECT|INSERT|UPDATE|DELETE|FROM|WHERE|JOIN|ORDER BY|GROUP BY|HAVING|LIMIT)\s+.*?;"
java_stack_pattern = r"(Exception in thread \"[^\"]*\"|at\s+[a-zA-Z0-9_.]+\s*$.*?$)"
def clean_text(text):
    # 清理 SQL 代码片段
    text = re.sub(sql_pattern, '', text, flags=re.IGNORECASE)
    # 清理 Java 异常堆栈
    text = re.sub(java_stack_pattern, '', text, flags=re.IGNORECASE)
    return text

# 过滤无意义的句子
#meaningful_sentences = filter_sentences(sentences, classifier)
#cleaned_description = ' '.join(meaningful_sentences)
#print("进一步清理后的描述内容：")
#print(cleaned_description)

# 提取关键词和特征
def extract_keywords(text):
    # 分词
    words = word_tokenize(text)
    
    # 移除停用词
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word.lower() not in stop_words and word.isalnum()]
    
    return filtered_words

# 提取特征
#keywords = extract_keywords(cleaned_description)
#print("提取的关键词：")
#print(keywords)

# 使用TF-IDF向量化文本内容
#vectorizer = TfidfVectorizer()
#tfidf_matrix = vectorizer.fit_transform([cleaned_description])
#print("TF-IDF特征矩阵：")
#print(tfidf_matrix.toarray())


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:

clean_text(description2)

"\nHello team,\n\n\n\nCould you help us to check the TLA Serial Count Report? The ‚ÄúMonths‚Äú value is being displayed as 2024, instead of the past months:\n\n!image-20240110-203955.png|width=827,height=578!\n\n\n\n\n\nThe query used is taking the current year to add it:\n\n\n\nmesr_aws=# select  \ndate_trunc('month',now()) ||'to'|| date_trunc('month',now() + INTERVAL '1 months') - INTERVAL '1 second' as sort_\n,to_char( (now() ), 'Month') || ' ' || extract( year from (now() )) as date_\nunion all\nselect  \ndate_trunc('month',now() - INTERVAL '1 months') ||'to'|| date_trunc('month',now() + INTERVAL '0 months') - INTERVAL '1 second'\n,to_char((now() - INTERVAL '1 months' ), 'Month') || ' ' || extract( year from (now() )) as date_\nunion all\nselect  \ndate_trunc('month',now() - INTERVAL '2 months') ||'to'|| date_trunc('month',now() - INTERVAL '1 months') - INTERVAL '1 second'\n,to_char((now() - INTERVAL '2 months' ), 'Month') || ' ' || extract( year from (now() )) as date_\nunion all\